# Geospatial Matching Optimization

This project is designed to showcase how to improve the speed of a polygon matching procedure. There are two main areas that could benefit from taking a "geospatial-native" approach:

1. Data encoding, fetching, and storage
2. Matching

We will generate synthetic polygon data and use both a non-geospatial-native and geospatial-native approach, comparing execution time for both to see how they differ.

## Imports

In [904]:
import geopandas as gpd
import pandas as pd
import shapely
import h3
import time
import helpers as h
import matplotlib.pyplot as plt
from shapely.wkt import dumps, loads
from pathlib import Path
import multiprocessing as mp

# ensure helpers is loaded correctly
import importlib
importlib.reload(h)

<module 'helpers' from '/Users/sra/files/projects/matching_optimization/helpers.py'>

## Retrieve Generated Test Data

In [905]:
prev_month_blobs = gpd.read_parquet('data/prev_month_blobs.parquet')
curr_month_blobs = gpd.read_parquet('data/curr_month_blobs.parquet')

In [906]:
# save to geojson to confirm in QGIS
prev_month_blobs.to_file('data/prev_month_blobs.geojson')
curr_month_blobs.to_file('data/curr_month_blobs.geojson')

In [907]:
prev_month_blobs.head()

,id,geohash,geometry
0,a9d7f295-582c-4938-9500-e3615bc76867,dhy2muq,"POLYGON ((-80.94975 26.78665, -80.94889 26.786..."
1,da13b533-be55-4b89-8125-d4d4b7990e63,dhy0wdc,"POLYGON ((-81.27473 26.82183, -81.27441 26.821..."
2,7a8688b2-bf42-423e-ac7a-35bfdb804d91,dhy2dne,"POLYGON ((-81.11798 26.84229, -81.11785 26.842..."
3,ea60ba8d-aafa-4462-9a94-31596a0b64ed,dhy0ex0,"POLYGON ((-81.40886 26.84562, -81.40851 26.845..."
4,5cfe6210-afc9-4dcb-a105-c5809e14ee53,dhy0v0w,"POLYGON ((-81.33433 26.85390, -81.33386 26.853..."


In [908]:
prev_month_blobs = prev_month_blobs[['geometry', 'id', 'geohash']]
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-80.94975 26.78665, -80.94889 26.786...",a9d7f295-582c-4938-9500-e3615bc76867,dhy2muq
1,"POLYGON ((-81.27473 26.82183, -81.27441 26.821...",da13b533-be55-4b89-8125-d4d4b7990e63,dhy0wdc
2,"POLYGON ((-81.11798 26.84229, -81.11785 26.842...",7a8688b2-bf42-423e-ac7a-35bfdb804d91,dhy2dne
3,"POLYGON ((-81.40886 26.84562, -81.40851 26.845...",ea60ba8d-aafa-4462-9a94-31596a0b64ed,dhy0ex0
4,"POLYGON ((-81.33433 26.85390, -81.33386 26.853...",5cfe6210-afc9-4dcb-a105-c5809e14ee53,dhy0v0w


In [909]:
curr_month_blobs = curr_month_blobs[['geometry', 'id', 'geohash']]
curr_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-81.45794 26.77572, -81.45720 26.775...",2f137277-29c8-481d-82bd-8b4553e099e6,dhy066k
1,"POLYGON ((-80.84526 26.77591, -80.84498 26.775...",a28b7d7e-92d0-4f49-85ea-6765206f1997,dhy8266
2,"POLYGON ((-81.18004 26.77972, -81.17932 26.779...",3dec7f9b-e912-4177-b9f3-dcc746ba91db,dhy22en
3,"POLYGON ((-81.06332 26.78989, -81.06249 26.789...",6965e327-a5d5-439e-924e-f3e9f95b608d,dhy27m5
4,"POLYGON ((-81.45134 26.79843, -81.45036 26.798...",0f896788-216c-4556-bc0d-cedcffe673cb,dhy06w9


In [910]:
# make data folder
Path('data/').mkdir(exist_ok=True)

# save as geoparquet
prev_month_blobs.to_parquet('data/prev_month_blobs.parquet')
curr_month_blobs.to_parquet('data/curr_month_blobs.parquet')

In [911]:
# range_ = 10000

# fig, ax = plt.subplots(figsize=(5, 5))
# prev_month_blobs.iloc[0:range_].plot(ax=ax)
# ax.set_title(f'Viewing the first {range_} blobs')
# plt.tight_layout()

# plt.show()

In [912]:
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-80.94975 26.78665, -80.94889 26.786...",a9d7f295-582c-4938-9500-e3615bc76867,dhy2muq
1,"POLYGON ((-81.27473 26.82183, -81.27441 26.821...",da13b533-be55-4b89-8125-d4d4b7990e63,dhy0wdc
2,"POLYGON ((-81.11798 26.84229, -81.11785 26.842...",7a8688b2-bf42-423e-ac7a-35bfdb804d91,dhy2dne
3,"POLYGON ((-81.40886 26.84562, -81.40851 26.845...",ea60ba8d-aafa-4462-9a94-31596a0b64ed,dhy0ex0
4,"POLYGON ((-81.33433 26.85390, -81.33386 26.853...",5cfe6210-afc9-4dcb-a105-c5809e14ee53,dhy0v0w


## 1. Non-optimized process

### 1.1. Data encoding, storage, and fetching

We will use PostgreSQL and string storage for geographic information.

#### Make copy of polygon layer and convert to non-geospatial-native string (WKT) datatype

In [913]:
# copy the gdfs
prev_month_blobs_wkt = prev_month_blobs.copy()
curr_month_blobs_wkt = curr_month_blobs.copy()

In [914]:
# Convert WKT versions to strings
dfs_to_convert = [prev_month_blobs_wkt, curr_month_blobs_wkt]
prev_month_blobs_wkt, curr_month_blobs_wkt = [h.convert_col_to_string(df) for df in dfs_to_convert]

# Check result
print(prev_month_blobs_wkt.head())

/Users/sra/files/projects/matching_optimization/helpers.py:319: UserWarning: Geometry column does not contain geometry.
  list: A list of tuples representing the rows of the dataframe.


Function `convert_col_to_string` executed in 0.5009 sec, CPU: 17.40%, Memory: 34.12MB
Function `convert_col_to_string` executed in 0.4905 sec, CPU: 38.30%, Memory: 31.17MB
                                            geometry  \
0  POLYGON ((-80.949755 26.786653, -80.948893 26....   
1  POLYGON ((-81.274734 26.821833, -81.274409 26....   
2  POLYGON ((-81.117975 26.842293, -81.117855 26....   
3  POLYGON ((-81.408859 26.845622, -81.408505 26....   
4  POLYGON ((-81.334326 26.8539, -81.333859 26.85...   

                                     id  geohash  
0  a9d7f295-582c-4938-9500-e3615bc76867  dhy2muq  
1  da13b533-be55-4b89-8125-d4d4b7990e63  dhy0wdc  
2  7a8688b2-bf42-423e-ac7a-35bfdb804d91  dhy2dne  
3  ea60ba8d-aafa-4462-9a94-31596a0b64ed  dhy0ex0  
4  5cfe6210-afc9-4dcb-a105-c5809e14ee53  dhy0v0w  


/Users/sra/files/projects/matching_optimization/helpers.py:319: UserWarning: Geometry column does not contain geometry.
  list: A list of tuples representing the rows of the dataframe.


#### Save in PostgreSQL database

This is a simple version of what a "non-geospatial-native" data ingestion pipeline may look like, with the primary example being that the polygons are stored as strings, not in a spatial-friendly datatype.

In [915]:
# Convert to tuples
prev_month_blobs_wkt = h.df_itertuple(prev_month_blobs_wkt)
curr_month_blobs_wkt = h.df_itertuple(curr_month_blobs_wkt)

Function `df_itertuple` executed in 0.0184 sec, CPU: 36.80%, Memory: 4.19MB
Function `df_itertuple` executed in 0.0148 sec, CPU: 57.10%, Memory: 3.52MB


Create PostgreSQL database if it doesn't exist already. We will be using the default settings. If they need to be adjusted, refer to [`helpers.py`](helpers.py).

In [916]:
h.create_pg_db()

Database blob_matching already exists.
Function `create_pg_db` executed in 0.1007 sec, CPU: 35.70%, Memory: 42.95MB


Create and insert into tables:

In [917]:
prev_month_blobs.head(1)

,geometry,id,geohash
0,"POLYGON ((-80.94975 26.78665, -80.94889 26.786...",a9d7f295-582c-4938-9500-e3615bc76867,dhy2muq


In [918]:
prev_month_blobs_wkt[0]

('POLYGON ((-80.949755 26.786653, -80.948893 26.786653, -80.948893 26.78716, -80.949755 26.78716, -80.949755 26.786653))',
 'a9d7f295-582c-4938-9500-e3615bc76867',
 'dhy2muq')

In [919]:
h.create_pg_table(table_name='prev_blobs_wkt', data=prev_month_blobs_wkt, truncate=True)
h.create_pg_table(table_name='curr_blobs_wkt', data=curr_month_blobs_wkt, truncate=True)

Table prev_blobs_wkt truncated.


Inserted 100000 records into prev_blobs_wkt.
Function `create_pg_table` executed in 4.4016 sec, CPU: 18.90%, Memory: 83.92MB
Table curr_blobs_wkt truncated.
Inserted 100000 records into curr_blobs_wkt.
Function `create_pg_table` executed in 3.9964 sec, CPU: 7.20%, Memory: 208.91MB


Retrieve data as GeoDataFrames to confirm that it worked:

In [920]:
df_prev = h.retrieve_pg_table(table_name='prev_blobs_wkt')
df_curr = h.retrieve_pg_table(table_name='curr_blobs_wkt')

Retrieved 100000 records from prev_blobs_wkt.
Function `retrieve_pg_table` executed in 0.6174 sec, CPU: 1.60%, Memory: 25.44MB
Retrieved 100000 records from curr_blobs_wkt.
Function `retrieve_pg_table` executed in 0.4320 sec, CPU: 34.40%, Memory: 62.45MB


Compare the tables before and after for a sanity check:

In [921]:
df_prev.head()

,geometry,id,geohash
0,"POLYGON ((-80.949755 26.786653, -80.948893 26....",a9d7f295-582c-4938-9500-e3615bc76867,dhy2muq
1,"POLYGON ((-81.274734 26.821833, -81.274409 26....",da13b533-be55-4b89-8125-d4d4b7990e63,dhy0wdc
2,"POLYGON ((-81.117975 26.842293, -81.117855 26....",7a8688b2-bf42-423e-ac7a-35bfdb804d91,dhy2dne
3,"POLYGON ((-81.408859 26.845622, -81.408505 26....",ea60ba8d-aafa-4462-9a94-31596a0b64ed,dhy0ex0
4,"POLYGON ((-81.334326 26.8539, -81.333859 26.85...",5cfe6210-afc9-4dcb-a105-c5809e14ee53,dhy0v0w


In [922]:
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-80.94975 26.78665, -80.94889 26.786...",a9d7f295-582c-4938-9500-e3615bc76867,dhy2muq
1,"POLYGON ((-81.27473 26.82183, -81.27441 26.821...",da13b533-be55-4b89-8125-d4d4b7990e63,dhy0wdc
2,"POLYGON ((-81.11798 26.84229, -81.11785 26.842...",7a8688b2-bf42-423e-ac7a-35bfdb804d91,dhy2dne
3,"POLYGON ((-81.40886 26.84562, -81.40851 26.845...",ea60ba8d-aafa-4462-9a94-31596a0b64ed,dhy0ex0
4,"POLYGON ((-81.33433 26.85390, -81.33386 26.853...",5cfe6210-afc9-4dcb-a105-c5809e14ee53,dhy0v0w


Confirm that the original blob dfs and the retrieved dfs are the same:

In [923]:
def round_geometry(geom, precision=6):
    """Round all coordinates of a geometry to a given precision."""
    return shapely.wkt.loads(shapely.wkt.dumps(geom, rounding_precision=precision))

# Convert both to sets of rounded WKT strings
set_prev_month_blobs = set(prev_month_blobs['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))
set_df_prev = set(df_prev['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))

# Find common, missing, and extra geometries
common_geometries = set_prev_month_blobs & set_df_prev
missing_from_retrieved = set_prev_month_blobs - set_df_prev
extra_in_retrieved = set_df_prev - set_prev_month_blobs

# Print summary
print(f"Number of matching geometries: {len(common_geometries)}")
print(f"Missing geometries in retrieved table: {len(missing_from_retrieved)}")
print(f"Extra geometries in retrieved table: {len(extra_in_retrieved)}")

# Show an example missing/extra geometry for debugging
if missing_from_retrieved:
    print("Example missing record:", next(iter(missing_from_retrieved)))

if extra_in_retrieved:
    print("Example extra record:", next(iter(extra_in_retrieved)))

Number of matching geometries: 100000
Missing geometries in retrieved table: 0
Extra geometries in retrieved table: 0


It worked! Now we can proceed to "matching" any "previous" polygons that overlap with the "current" polygons.

### 1.2. Matching

We will match the polygons using GeoPandas.

In [924]:
postgresql_details = h.pg_details()
h.run_parallel_matching(table_prev='prev_blobs_wkt', 
                        table_curr='curr_blobs_wkt', 
                        output_table='matched_results', 
                        postgresql_details=postgresql_details, 
                        db_name='blob_matching', 
                        num_workers=4, 
                        batch_size=100)

Table matched_results created successfully.
Retrieved 100000 records from prev_blobs_wkt.
Function `match_geometries` executed in 0.0047 sec, CPU: 5.90%, Memory: 3.67MB
Function `match_geometries` executed in 0.0051 sec, CPU: 87.30%, Memory: 7.98MB
Function `match_geometries` executed in 0.0064 sec, CPU: 1.00%, Memory: 1.44MB
Function `match_geometries` executed in 0.0051 sec, CPU: 22.40%, Memory: 1.22MB
Function `match_geometries` executed in 0.0052 sec, CPU: 37.90%, Memory: 1.70MB
Function `match_geometries` executed in 0.0050 sec, CPU: 69.10%, Memory: 0.95MB
Function `match_geometries` executed in 0.0051 sec, CPU: 63.80%, Memory: 0.53MB
Function `match_geometries` executed in 0.0050 sec, CPU: 40.10%, Memory: 1.55MB
Function `match_geometries` executed in 0.0057 sec, CPU: 17.90%, Memory: 3.72MB
Function `match_geometries` executed in 0.0055 sec, CPU: 67.80%, Memory: 0.00MB
Function `match_geometries` executed in 0.0049 sec, CPU: 11.10%, Memory: 3.28MB
Function `match_geometries` exec

Process Process-2694:
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sra/files/projects/matching_optimization/helpers.py", line 220, in <module>
    admin_boundaries: gpd.GeoDataFrame = gpd.read_file('data/conus_admin.parquet'), 
                                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/site-packages/geopandas/io/file.py", line 289, in _read_file
    return _read_file_fiona(
           ^^^^^^^^^^^^^^^^^
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/pyth

KeyboardInterrupt: 

In [894]:
# # testing logging for matching_geometries
# df_prev = h._retrieve_pg_table(postgresql_details, 'blob_matching', "prev_blobs_wkt", log_enabled=True)
# df_curr = h._retrieve_pg_table(postgresql_details, 'blob_matching', "curr_blobs_wkt", log_enabled=True)

# # Use a small subset for testing
# df_prev_sample = df_prev.sample(10, random_state=42)
# df_curr_sample = df_curr.sample(10, random_state=42)

# h.match_geometries(df_prev_sample, df_curr_sample)  # Check if this logs matches

## 2. Optimized Approach

Having demonstrated a reasonable if not optimized way of matching polygons, we will now make use of geospatial-native tools to hopefully match polygons much more quickly.

My proposed workflow centers around `DuckDB` and `H3` along with the `.parquet` format to efficiently store, retrieve, and process geospatial data. We will use `Apache Iceberg` as a layer over the parquet file as it allows for more robust table handling.

1. Prepare GeodataFrames with H3 Polyfill and parent hexagons at lower resolution.
   * Save as an Iceberg table.

2. Use DuckDB and the H3 extension to execute spatial SQL queries to find overlapping polygons with H3-based joins.

3. Save matching polygon IDs to a new Iceberg table.

4. Visualize the results.

### 2.1. Add H3 data to the blobs

H3 is a modern and efficient geospatial index paradigm that divides the world into nested hexagons.

In [902]:
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-81.10504 26.01885, -81.04984 26.018...",6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0,dhwk5h
1,"POLYGON ((-80.91675 26.04190, -80.86358 26.041...",a0fbead2-d8dc-4e05-b364-ab89d020fbfb,dhwkpq
2,"POLYGON ((-80.73525 26.20304, -80.71903 26.203...",b4c64915-35eb-4d74-a211-ce41ad8f21f8,dhwt55
3,"POLYGON ((-80.86288 26.17613, -80.79088 26.176...",d03d2784-ff88-4ed6-b93c-2b52a8a7ecc3,dhwt0s
4,"POLYGON ((-81.28584 26.19397, -81.21442 26.193...",0ce2ca44-0727-4364-978b-da23cd9cd784,dhwjpj


In [903]:
def compute_h3_indices(geometry, centroid_res=6, polyfill_res=9):
    """
    Compute H3 indices for a polygon:
    - Single H3 index based on centroid
    - Full polygon coverage with H3 polyfill at high resolution
    """
    centroid = geometry.centroid
    h3_centroid = h3.geo_to_h3(centroid.y, centroid.x, centroid_res)

    # Full coverage using polyfill
    h3_polyfill = list(h3.polyfill(geometry.__geo_interface__, polyfill_res))

    return h3_centroid, h3_polyfill

In [ ]:
test_prev = prev_month_blobs.copy()
test_curr = curr_month_blobs.copy()

for gdf in [test_prev, test_curr]:
    gdf['h3_centroid'] = gdf['geometry'].apply(lambda geom: compute_h3_indices(geom, 6, 9))

In [899]:
print(type(prev_month_blobs))
prev_month_blobs.head(3)
# prev_month_blobs = gpd.read_parquet('data/prev_month_blobs.parquet')
# curr_month_blobs = gpd.read_parquet('data/curr_month_blobs.parquet')

<class 'geopandas.geodataframe.GeoDataFrame'>


,geometry,id,geohash
0,"POLYGON ((-81.10504 26.01885, -81.04984 26.018...",6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0,dhwk5h
1,"POLYGON ((-80.91675 26.04190, -80.86358 26.041...",a0fbead2-d8dc-4e05-b364-ab89d020fbfb,dhwkpq
2,"POLYGON ((-80.73525 26.20304, -80.71903 26.203...",b4c64915-35eb-4d74-a211-ce41ad8f21f8,dhwt55


# Notes on original process

geohash polygon into a larger region
match blobs using multiprocessing (mp.Process) and geopandas
find blobs from previous month that do not have a match

# 1 imports and setup

```python
import os, sys, traceback
import argparse
import cv2
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io, color, measure
from tqdm import tqdm
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.validation import make_valid
import PIL
from PIL import Image, ImageDraw, ImageEnhance
import uuid
import multiprocessing as mp
from datetime import datetime
from sqlalchemy import or_
from sqlmodel import SQLModel, Session, create_engine, select
from pathlib import Path
```

## Key takeaways
- Uses Pandas and GeoPandas for data processing.
- Uses Shapely for geospatial geometry operations.
- Uses multiprocessing (mp) to parallelize the blob-matching process.
- Uses SQLAlchemy and SQLModel for database operations.

# 2 High-level script overview
The script is designed to match blobs (spatial objects) between two months and classify them into different business categories.

General Workflow
1. Fetch blobs from the previous and current months.
1. Convert the region of interest (state, city, county, or geohash) into a list of geohashes.
1. Parallelized Matching Process:
1. Process multiple geohashes at once using multiprocessing.
1. Find corresponding blobs from previous months for each geohash.
1. Classify blobs based on construction stage progression.
1. Identify blobs that are missing in the current month and "impute" them.
1. Save results into the database.

# 3 Blob classification logic

# 4 Blob matching

- Matches blobs between months using polygon intersections.
- Uses Shapely to validate geometries and check for overlaps.
- Returns matched blob IDs.

## Multiprocessing Optimization
The script parallelizes blob matching by:
- Uses multiprocessing (mp.Process) to divide the dataset into smaller batches.
- Each batch of blobs is processed in parallel.
- Reduces runtime compared to a single-threaded approach.

# 5 Handling unmatched blobs
- Finds blobs from the previous month that do not have a match in the current month.
- These blobs are "imputed", meaning they are carried over into the new month.

# 6 Main class to orchestrate process
- Loads previous and current month blobs.
- Converts input regions (state, city, county) into geohashes.

# Summary
- Parallel processing is used to match blobs.
- Blobs are classified based on construction stages.
- Unmatched blobs are imputed for continuity.
- The script writes results to a database.